In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('C://Users//akhil//Desktop//HAP 880//Week 2//testclaims_hu.db')

In [3]:
df = pd.read_sql('select * from highUtilizationPredictionV2wco', conn)

In [4]:
df=df.join(pd.get_dummies(df.race))

In [5]:
df.head()

,index,race,age,patient_id,ELIX1,ELIX2,ELIX3,ELIX4,ELIX5,ELIX6,...,drugs_m11-12,HighUtilizationY2,claimCount,A,Am.N,B,H,O,U,W
0,0,B,71,PAT136597,0,0,0,0,0,1,...,1,1,160,0,0,1,0,0,0,0
1,1,A,86,PAT119838,0,0,0,0,0,0,...,0,0,24,1,0,0,0,0,0,0
2,2,W,70,PAT11289,1,0,0,0,0,0,...,0,0,52,0,0,0,0,0,0,1
3,3,W,75,PAT178745,0,0,0,0,1,0,...,0,0,15,0,0,0,0,0,0,1
4,4,W,77,PAT50922,0,0,0,0,1,0,...,4,0,66,0,0,0,0,0,0,1


In [6]:
cols=list(df.columns)

In [7]:
cols.remove('index')
cols.remove('race')
cols.remove('patient_id')
cols.remove('HighUtilizationY2')
cols.remove('claimCount')

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
tr, ts = train_test_split(df)

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

In [11]:
lr = LogisticRegression()

In [12]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
params={"C": [1.0,0.8,0.5,0.3,0.01], "solver":['liblinear','lbfgs'], "class_weight":[None, 'balanced']}

In [15]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

In [16]:
res_lr = []
for c in params["C"]:
    lr = LogisticRegression(C=c, solver='lbfgs', max_iter=100)
    scores = [c, cross_validate(lr, tr[cols], tr['HighUtilizationY2'], scoring=['roc_auc','accuracy','precision','recall'], cv=10)]
    res_lr.append(scores)

In [17]:
res_lr

C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecatio

[[1.0,
  {'fit_time': array([1.05224586, 0.96600151, 0.89401197, 0.9380424 , 0.94367051,
          1.08036065, 1.25582385, 1.32149243, 1.25019145, 1.07657003]),
   'score_time': array([0.02792478, 0.0259304 , 0.02991772, 0.02593088, 0.02792525,
          0.03291941, 0.02792597, 0.02892447, 0.03390908, 0.02892065]),
   'test_roc_auc': array([0.81666292, 0.82883554, 0.80197104, 0.79311171, 0.8215035 ,
          0.82390593, 0.80202993, 0.81436264, 0.81400188, 0.8286222 ]),
   'train_roc_auc': array([0.81553557, 0.81818532, 0.8159722 , 0.81421175, 0.81569027,
          0.81277996, 0.8172362 , 0.81564582, 0.81723371, 0.82007311]),
   'test_accuracy': array([0.93571597, 0.9348903 , 0.93677008, 0.93547245, 0.93830365,
          0.93747788, 0.93723454, 0.93829637, 0.93640868, 0.93723454]),
   'train_accuracy': array([0.93700354, 0.93733124, 0.93727963, 0.93731895, 0.93683396,
          0.93718787, 0.93712316, 0.93697898, 0.93690033, 0.93700519]),
   'test_precision': array([0.52873563, 0.48913

In [18]:
lr.get_params()

{'C': 0.01,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [19]:
res_df = pd.DataFrame(columns=["C","solver","score","score_val0","score_val1","score_val2","score_val3","score_val4","score_val5","score_val6","score_val7","score_val8","score_val9",])

In [21]:
cnt=0
for r in res_lr:
    for k,v in r[1].items():
      l = [r[0]]
      l.append('lbfgs')
      l.append(k)
      for i in v:
            l.append(i)
      res_df.loc[cnt]=l  
      cnt = cnt + 1

In [22]:
res_df

,C,solver,score,score_val0,score_val1,score_val2,score_val3,score_val4,score_val5,score_val6,score_val7,score_val8,score_val9
0,1.00,lbfgs,fit_time,1.052246,0.966002,0.894012,0.938042,0.943671,1.080361,1.255824,1.321492,1.250191,1.076570
1,1.00,lbfgs,score_time,0.027925,0.025930,0.029918,0.025931,0.027925,0.032919,0.027926,0.028924,0.033909,0.028921
2,1.00,lbfgs,test_roc_auc,0.816663,0.828836,0.801971,0.793112,0.821504,0.823906,0.802030,0.814363,0.814002,0.828622
3,1.00,lbfgs,train_roc_auc,0.815536,0.818185,0.815972,0.814212,0.815690,0.812780,0.817236,0.815646,0.817234,0.820073
4,1.00,lbfgs,test_accuracy,0.935716,0.934890,0.936770,0.935472,0.938304,0.937478,0.937235,0.938296,0.936409,0.937235
5,1.00,lbfgs,train_accuracy,0.937004,0.937331,0.937280,0.937319,0.936834,0.937188,0.937123,0.936979,0.936900,0.937005
6,1.00,lbfgs,test_precision,0.528736,0.489130,0.554622,0.509091,0.622642,0.606742,0.576577,0.625000,0.546296,0.602410
7,1.00,lbfgs,train_precision,0.569106,0.588364,0.585129,0.580581,0.563136,0.581270,0.576681,0.582017,0.574053,0.584882
8,1.00,lbfgs,test_recall,0.083636,0.081818,0.120219,0.102004,0.120219,0.098361,0.116576,0.118397,0.107468,0.091075
9,1.00,lbfgs,train_recall,0.113314,0.108458,0.109852,0.117338,0.111875,0.109245,0.111066,0.096905,0.101153,0.095489


In [23]:
res_df['average'] = res_df[["score_val0","score_val1","score_val2","score_val3","score_val4","score_val5","score_val6","score_val7","score_val8","score_val9"]].mean(numeric_only=True, axis=1)

In [24]:
res_df

,C,solver,score,score_val0,score_val1,score_val2,score_val3,score_val4,score_val5,score_val6,score_val7,score_val8,score_val9,average
0,1.00,lbfgs,fit_time,1.052246,0.966002,0.894012,0.938042,0.943671,1.080361,1.255824,1.321492,1.250191,1.076570,1.077841
1,1.00,lbfgs,score_time,0.027925,0.025930,0.029918,0.025931,0.027925,0.032919,0.027926,0.028924,0.033909,0.028921,0.029023
2,1.00,lbfgs,test_roc_auc,0.816663,0.828836,0.801971,0.793112,0.821504,0.823906,0.802030,0.814363,0.814002,0.828622,0.814501
3,1.00,lbfgs,train_roc_auc,0.815536,0.818185,0.815972,0.814212,0.815690,0.812780,0.817236,0.815646,0.817234,0.820073,0.816256
4,1.00,lbfgs,test_accuracy,0.935716,0.934890,0.936770,0.935472,0.938304,0.937478,0.937235,0.938296,0.936409,0.937235,0.936780
5,1.00,lbfgs,train_accuracy,0.937004,0.937331,0.937280,0.937319,0.936834,0.937188,0.937123,0.936979,0.936900,0.937005,0.937096
6,1.00,lbfgs,test_precision,0.528736,0.489130,0.554622,0.509091,0.622642,0.606742,0.576577,0.625000,0.546296,0.602410,0.566124
7,1.00,lbfgs,train_precision,0.569106,0.588364,0.585129,0.580581,0.563136,0.581270,0.576681,0.582017,0.574053,0.584882,0.578522
8,1.00,lbfgs,test_recall,0.083636,0.081818,0.120219,0.102004,0.120219,0.098361,0.116576,0.118397,0.107468,0.091075,0.103977
9,1.00,lbfgs,train_recall,0.113314,0.108458,0.109852,0.117338,0.111875,0.109245,0.111066,0.096905,0.101153,0.095489,0.107470


In [44]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
res_rf = []
for t in list(map(int,np.linspace(10, 100, num=3)))+list(map(int,np.linspace(200, 1000, num=2))):
    for f in ["sqrt","log2",None]:
      print('Running RF: ', t, f)
      rf = RandomForestClassifier(n_estimators = t, max_features = f)
      scores = [t, f, cross_validate(rf, tr[cols], tr['HighUtilizationY2'], scoring=['roc_auc','accuracy','precision','recall'], cv=10)]
      res_rf.append(scores)

Running RF:  10 sqrt
Running RF:  10 log2
Running RF:  10 None
Running RF:  55 sqrt
Running RF:  55 log2
Running RF:  55 None
Running RF:  100 sqrt
Running RF:  100 log2
Running RF:  100 None
Running RF:  200 sqrt
Running RF:  200 log2
Running RF:  200 None
Running RF:  1000 sqrt
Running RF:  1000 log2
Running RF:  1000 None


In [45]:
res_rf

C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecatio

[[10,
  'sqrt',
  {'fit_time': array([1.08280468, 0.98315859, 1.05116796, 0.96878433, 1.07296205,
          1.25250459, 1.0642283 , 1.0701232 , 1.02904677, 1.02408957]),
   'score_time': array([0.11467409, 0.09915996, 0.10468006, 0.10599256, 0.10172772,
          0.10073447, 0.09719014, 0.10925341, 0.11868382, 0.11089134]),
   'test_roc_auc': array([0.7291901 , 0.7518441 , 0.73871871, 0.73490559, 0.73733662,
          0.74692623, 0.74391321, 0.74362966, 0.74072887, 0.74514048]),
   'train_roc_auc': array([0.99872323, 0.99851931, 0.99896056, 0.99857838, 0.99859336,
          0.99856636, 0.99861391, 0.99875491, 0.99869569, 0.99872878]),
   'test_accuracy': array([0.93536211, 0.93606983, 0.93818568, 0.93441076, 0.93417483,
          0.93783178, 0.93770647, 0.93558282, 0.9362907 , 0.93640868]),
   'train_accuracy': array([0.98864858, 0.98866169, 0.98919925, 0.98896331, 0.98883223,
          0.98905507, 0.98919939, 0.98931736, 0.98874063, 0.98950087]),
   'test_precision': array([0.51111111

In [46]:
import pickle

In [47]:
with open('res_rf1.pkl', 'wb') as f:
    pickle.dump(res_rf, f)

In [48]:
with open('res_rf1.pkl', 'rb') as f:
    res_rf2 = pickle.load(f)

In [49]:
res_rf2

C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_precision'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\akhil\Anaconda3\lib\site-packages\sklearn\utils\deprecatio

[[10,
  'sqrt',
  {'fit_time': array([1.08280468, 0.98315859, 1.05116796, 0.96878433, 1.07296205,
          1.25250459, 1.0642283 , 1.0701232 , 1.02904677, 1.02408957]),
   'score_time': array([0.11467409, 0.09915996, 0.10468006, 0.10599256, 0.10172772,
          0.10073447, 0.09719014, 0.10925341, 0.11868382, 0.11089134]),
   'test_roc_auc': array([0.7291901 , 0.7518441 , 0.73871871, 0.73490559, 0.73733662,
          0.74692623, 0.74391321, 0.74362966, 0.74072887, 0.74514048]),
   'train_roc_auc': array([0.99872323, 0.99851931, 0.99896056, 0.99857838, 0.99859336,
          0.99856636, 0.99861391, 0.99875491, 0.99869569, 0.99872878]),
   'test_accuracy': array([0.93536211, 0.93606983, 0.93818568, 0.93441076, 0.93417483,
          0.93783178, 0.93770647, 0.93558282, 0.9362907 , 0.93640868]),
   'train_accuracy': array([0.98864858, 0.98866169, 0.98919925, 0.98896331, 0.98883223,
          0.98905507, 0.98919939, 0.98931736, 0.98874063, 0.98950087]),
   'test_precision': array([0.51111111

In [ ]:
from skopt.space import Integer, Categorical, Real
from skopt.utils import use_named_args
from skopt import gp_minimize

In [ ]:
space = [Integer(10, 60, name='n_estimators'),
         Categorical(['sqrt', 'log2'], name='max_features'),
         Categorical(['gini','entropy'], name='criterion')]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
@use_named_args(space)
def objective(**params):
    rf = RandomForestClassifier(**params)
    return -np.mean(cross_val_score(rf, tr[cols], tr['HighUtilizationY2'], cv=5, n_jobs=1, scoring='roc_auc'))

In [ ]:
reg_gp = gp_minimize(objective, space, verbose=True)